In [1]:
import pandas as pd

# Exemplo de DataFrame
data = {
    'PartnerID': [1, 1, 2, 3, 3, 3],
    'Acceptance Date': ['2024-12-01', '2024-11-01', '2024-10-01', '2024-12-20', '2024-11-20', '2024-10-15'],
    'Contract': ['C1', 'C2', 'C3', 'C4', 'C5', 'C6']
}
df = pd.DataFrame(data)

# Converter a coluna de datas para o formato datetime
df['Acceptance Date'] = pd.to_datetime(df['Acceptance Date'])

# Identificar PartnerID duplicados
duplicated_partner_ids = df[df.duplicated('PartnerID', keep=False)]

# Obter o contrato com a data mais recente para cada PartnerID duplicado
most_recent_contracts = duplicated_partner_ids.loc[
    duplicated_partner_ids.groupby('PartnerID')['Acceptance Date'].idxmax()
]

# Resultado final
print(most_recent_contracts)


   PartnerID Acceptance Date Contract
0          1      2024-12-01       C1
3          3      2024-12-20       C4


In [3]:
duplicated_partner_ids.groupby('PartnerID')['Acceptance Date'].idxmax()


PartnerID
1   2024-12-01
3   2024-12-20
Name: Acceptance Date, dtype: datetime64[ns]

In [42]:
path = 'C:\\Users\\edinocencio\\DataClenupCmsCrm\\DataSources\\'
file_name = 'PMD-CMS.xlsx'
full_file_name = path + file_name
sheet_name = 'after synch'

df_master = pd.read_excel(full_file_name, sheet_name=sheet_name)

HEADERS = df_master.iloc[0, :].tolist()

df_master.columns = HEADERS
df_master = df_master.loc[1:,['PartnerID', 'Branch','ContractNumber']]

df_master['CONTRACT'] = df_master['Branch'].astype(str) + "-" + df_master['ContractNumber'].astype(str)

df_getinvoice = pd.read_excel('DataSources\\GetInvoice.xlsx')
df_getinvoice = df_getinvoice[['B+C','Acceptance Date']]

df_master = df_master.merge(df_getinvoice,left_on='CONTRACT', right_on='B+C', how='left')


In [44]:
df_master['Acceptance Date'] = pd.to_datetime(df_master['Acceptance Date'])

In [45]:
# Identificar PartnerID duplicados
duplicated_partner_ids = df_master[
    df_master.duplicated(subset='PartnerID', keep=False)]['PartnerID'].unique()

In [48]:
# Criar DataFrame com PartnerIDs duplicados
df_duplicated = df_master[df_master['PartnerID'].isin(duplicated_partner_ids)]


In [49]:
# Identificar contratos mais recentes para PartnerIDs duplicados
df_recent_contracts = df_duplicated.loc[
    df_duplicated.groupby('PartnerID')['Acceptance Date'].idxmax()
]

In [53]:
# Criar DataFrame com PartnerIDs únicos
df_unique = df_master[~df_master['PartnerID'].isin(duplicated_partner_ids)]

# Salvar resultados em arquivos separados
df_recent_contracts.to_excel('DataSources\\finalizados\\recent_contracts.xlsx', index=False)
df_unique.to_excel('DataSources\\finalizados\\unique_partner_ids.xlsx', index=False)

print("Os contratos mais recentes foram salvos em 'recent_contracts.xlsx'.")
print("Os PartnerIDs únicos foram salvos em 'unique_partner_ids.xlsx'.")

Os contratos mais recentes foram salvos em 'recent_contracts.xlsx'.
Os PartnerIDs únicos foram salvos em 'unique_partner_ids.xlsx'.
